In [ ]:
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

# Web Loader
%pip install -qU beautifulsoup4

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)

Compile data together

In [2]:
import pandas as pd

entries = pd.read_csv("processed.csv")

In [3]:
notes_compiled = ""


#full_date,date,weekday,time,mood,activities,note_title,note

for idx, entry in entries.iterrows():
    notes_compiled += f"""
    { str(entry['note']) if str(entry['note']) != "nan" else ""}
    """
    

In [4]:
pip install langchain
notes_compiled = notes_compiled.replace("<br>", ".")
notes_compiled = notes_compiled.replace("</br>", ".")

notes_compiled = notes_compiled.replace("<b>", ".")
notes_compiled = notes_compiled.replace("</b>", ".")


In [5]:
with open("Output.txt", "w") as text_file:
    text_file.write(notes_compiled)

In [6]:
all_splits = text_splitter.split_text(notes_compiled)

In [ ]:
len(all_splits)

In [8]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_texts(texts=all_splits, embedding=local_embeddings)

In [ ]:
question = "who am i as a person and what do i value?"
docs = vectorstore.similarity_search(question, k=25)
print(len(docs))
print(docs[0])

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
)

In [ ]:
from IPython.display import Markdown, display

response_message = model.invoke(
    "my name is ara"
)

# Compile the output to markdown
markdown_output =response_message.content

# Print or display the markdown output
display(Markdown(markdown_output))

response_message2 = model.invoke(
    "what is my name"
)

# Compile the output to markdown
markdown_output2 =response_message.content

# Print or display the markdown output
display(Markdown(markdown_output2))

In [25]:
# Handles the memory of the conversation history
class ChatMemory:
    def __init__(self):
        self.history = []
        
    #Add a new exchange (user question and assistant answer) to memory.
    def add_to_memory(self, user_input, assistant_output):
        self.history.append(f"User: {user_input}")
        self.history.append(f"Assistant: {assistant_output}")
        
    def get_context(self):
        # Return the full context (conversation history) as a single string
        return "\n".join(self.history)
    
    def clear_context(self):
        self.history = []



In [47]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

RAG_TEMPLATE = """
You are a helpful assistant who keeps track of all past exchanges. Here is your conversation history
<Conversation History>
{history}
</Conversation History>

Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. These are some entries at different dates that give insight into your state of mind, memories and moods. Use direct references when necessary
Be as specific and thorough as possible and prioritize conversation history when possible to be able to gain more context and quote the conversation history context when applicable
<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chat_memory = ChatMemory()

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    # | RunnableLambda(lambda formatted_prompt: print(f"Formatted Prompt:\n{formatted_prompt}") or formatted_prompt)
    | model
    | StrOutputParser()
)


In [20]:
def ask_question(question):
    context = chat_memory.get_context()
    
    # Get documents related to the current question
    docs = vectorstore.similarity_search(question, k=25)
    
    markdown_output = chain.invoke({"history": context, "context": docs, "question": question})
    
    # Add to the chat memory
    chat_memory.add_to_memory(question, markdown_output)
    
    # Return the response
    return markdown_output


In [ ]:
chat_memory.clear_context()

print("----------------")
print(chat_memory.get_context())
print("----------------")

question = "have i lost some of my hobbies over time?"
response = ask_question(question)

display(Markdown(response))

# print("----------------")
# print(chat_memory.get_context())
# print("----------------")

# follow_up_question = "can you tell me more about my concerns with him?"

# follow_up_response = ask_question(follow_up_question)
# display(Markdown(follow_up_response))

# print("----------------")
# print(chat_memory.get_context())
# print("----------------")


# # You can continue the conversation with more questions
# another_question = "How can I work on these feelings to be closer with him?"
# another_response = ask_question(another_question)
# display(Markdown(another_response))

# print("----------------")
# print(chat_memory.get_context())
# print("----------------")


In [ ]:

print("----------------")
print(chat_memory.get_context())
print("----------------")

question = "how can i work to get these hobbies back?"
response = ask_question(question)

display(Markdown(response))